# Extract raw RTTs from MongoDB

## Parameters

In [24]:
experiment_database = "micro1_ext"
output_folder = "micro1_ext2"
reg_to_catch = "xp1_"
raw_data_folder = "raw"
concat_data_folder = "concat"
input_RPS = ['375','500']

In [25]:
output_base_dir = "./raw_results/" + output_folder
raw_base_dir = output_base_dir + "/" + raw_data_folder
concat_base_dir = output_base_dir + "/" + concat_data_folder

## Create experiment subfolders

In [26]:
import os
if not os.path.exists(raw_base_dir):
    os.makedirs(raw_base_dir)
if not os.path.exists(concat_base_dir):
    os.makedirs(concat_base_dir)
for RPS in input_RPS:
    if not os.path.exists(raw_base_dir + "/" + RPS):
        os.makedirs(raw_base_dir + "/" + RPS)

## Get list of runs

In [27]:
from pymongo import MongoClient
import pymongo

client = MongoClient("manager-mongo-manager")
db = client[experiment_database]
coll_names = db.list_collection_names(nameOnly=True)
#print(coll_names)

## Output raw RTT into files

In [ ]:
import re
for coll_name in coll_names:
    if coll_name != "experiments":
        RPS = re.findall(r'%s(\d+)' % reg_to_catch, coll_name)[0]
        with open(raw_base_dir + "/" + RPS + "/" + coll_name + ".dat", "w") as f:
            cursor = db[coll_name].find()
            entries = list(cursor)
            for entry in entries:
                print(str(entry.get("response_time")), file=f)
print("Done!")

## Merge experiments

In [11]:
import os
for RPS in input_RPS:
    rps_base_dir = raw_base_dir + "/" + RPS
    with open(concat_base_dir + "/" + RPS + ".dat", "w") as fo:
        for r, d, fs in os.walk(rps_base_dir):
            for f in fs:
                if os.path.isfile(rps_base_dir + "/" + f):
                    with open(rps_base_dir + "/" + f,'r') as fi:
                        fo.write(fi.read())
print("Done!")

Done!
